# Notebook 6: Create Plots

The following notebook uses the dictionaries computed in notebook `5_compute_results.ipynb` (or the dictionaries in the `results_article` folder) to generate plots regarding CO2 emissions and route diversity

In [ ]:
import numpy as np
import json
import gzip
from matplotlib import pyplot as plt

def load_dict_from_gzipped_json(filename):
    with gzip.open(filename, 'rt') as gzipped_file:
        data = json.load(gzipped_file)
    return data

def get_measure(dict_city, navigator, base, N, measure, divide_by=1):

    percentages = np.arange(0, 101, 10)

    dict_measure = dict_city[navigator][base][N][measure]
    avg_measure = [np.mean(dict_measure[str(p)])/divide_by for p in percentages]
    std_measure = [np.std(dict_measure[str(p)])/divide_by for p in percentages]

    return avg_measure, std_measure

def apply_my_axes_style(ax):
    ax.grid(alpha=.2)
    ax.set_box_aspect(1)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

General Parameters

In [ ]:
city_to_N_max = {"milan": 100000, "florence": 50000, "rome":100000}
percentages = np.arange(0, 101, 10)

SERVICE2ACRONYM = {'bing': 'Bi', 
                 'mapbox': 'MB', 
                 'gmaps': 'GM', 
                'tomtomFastest': 'TTF', 
                   'tomtomShort': 'TTS', 
                 'tomtomEco': 'ecoTT',
                  'IGfastest':'FP',
                  "interplay": "INT"}

SERVICE2COLOR = {'bing': '#1f77b4', 
                 'mapbox': '#ff7f0e', 
                 'gmaps': '#2ca02c', 
                 'tomtomFastest': '#d62728', 
                 'tomtomShort': '#9467bd', 
                 'tomtomEco': '#8c564b',
                 'IGfastest':'blue',
                 'interplay':'black'}

### Load the dictionary containing the aggregated results

In [ ]:
path_dict_results = "../results_article/results_milan.json.gz"

dict_results = load_dict_from_gzipped_json(path_dict_results)

#### Info on the configuration to plot

In [ ]:
navigators_to_plot = ["tomtomFastest", "tomtomEco", "tomtomShort", "bing", "mapbox", "gmaps"]

base = "myduaw5"
N = 65000

# Route Diversity

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
measure = "traveled_edges"


for navigator in navigators_to_plot:

    avg_values_navigator, std_values_navigator = get_measure(dict_results, navigator, base, str(N), measure)
    
    
    ax.errorbar(x=percentages, y=avg_values_navigator, yerr=std_values_navigator, 
                        marker = "o", markersize=8, color=SERVICE2COLOR[navigator], linestyle=":",
                        linewidth=2, label=SERVICE2ACRONYM[navigator])

apply_my_axes_style(ax)

ax.set_ylabel("route diversity", fontsize=15)
ax.tick_params(axis='both', which='major', labelsize=15, length=7, width=2);
ax.set_xlabel('adoption rate $r$ (%)', fontsize=15);
plt.legend();

# CO2 emissions

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
measure = "total_co2"

express_in_kg = False
normalize_by_N = False


for navigator in navigators_to_plot:

    avg_values_navigator, std_values_navigator = get_measure(dict_results, navigator, base, str(N), measure)
    
    if "total_co2" in measure:
        if express_in_kg:
            avg_values_navigator = 1e3*np.array(avg_values_navigator)/1e9
            std_values_navigator = 1e3*np.array(std_values_navigator)/1e9
        else:
            avg_values_navigator = np.array(avg_values_navigator)/1e9
            std_values_navigator = np.array(std_values_navigator)/1e9

    if normalize_by_N:
        avg_values_navigator = np.array(avg_values_navigator)/N_measure
        std_values_navigator = np.array(std_values_navigator)/N_measure
    
    ax.errorbar(x=percentages, y=avg_values_navigator, yerr=std_values_navigator, 
                        marker = "o", markersize=8, color=SERVICE2COLOR[navigator], linestyle=":",
                        linewidth=2, label=SERVICE2ACRONYM[navigator])

apply_my_axes_style(ax)

if normalize_by_N:
    ax.set_ylabel("CO2 per vehicle (kg)", fontsize=15)
else:
    ax.set_ylabel("tot. CO2 (tons)", fontsize=15)

ax.tick_params(axis='both', which='major', labelsize=15, length=7, width=2);
ax.set_xlabel('adoption rate $r$ (%)', fontsize=15);

plt.legend();